In [1]:
from datetime import datetime
import mysql.connector

# estrablishing the connectiontomySQL as i have stored my data there
conn = mysql.connector.connect(host='localhost',
                              database='assign',
                              user='root',
                              password='Aaric@218')

In [2]:
# checking the connection status
conn.is_connected()

True

In [3]:
conn.connect()

In [4]:
# checking the status of databse connection
if conn.is_connected():
    print("DB CONNECTED")
    cursor = conn.cursor()
    cursor.execute("USE assign;")
    record = cursor.fetchone()
    print(record)

DB CONNECTED
None


In [13]:
## This is class to perform checking account balance, to perform withdraw of amount asked and fetch the transaction details

class assign:
    # A constructor that is called automatically when the class is called with userid as parameter
    def __init__(self,userid:int):
        self.userid = userid
       
    # function to check account balance of given userid
    def check_account_bal(self):
        conn.connect()
        curent_bal = cursor.callproc("account_bal",[self.userid,'@M'])[1]
        #conn.close()
        return curent_bal
    
    # Function to update account after withdraw
    def updateBank_account(self,withdraw_amount:int):
        current_bal = self.check_account_bal()
        update_bal = current_bal - withdraw_amount
        try:
            conn.connect()
            cursor.execute(f"UPDATE bank_account SET amount = '{update_bal}' WHERE UserID = '{self.userid}'")
            conn.commit()
            conn.close()
        except Exception as e:
            print(e)
    
    # function to check minimum balance and make withdraw:
    def withdraw(self,withdraw_amount:int):
        conn.connect()
        curent_bal = self.check_account_bal()
        status = cursor.callproc("withdraw_amt",[self.userid,'@M'])[1]
        conn.close()
        if status == None:
            print(f"You do not have sufficient balance\nCurrent balance :: {curent_bal}")
        else:
            new_bal = curent_bal - withdraw_amount
            if new_bal > 5000:
                self.updateBank_account(withdraw_amount)
                date = datetime.today().date()
                self.log_transaction(date,withdraw_amount)
                print(f"Transaction successfull!!!\nCurrent bal :: {new_bal}")
            else:
                print(f"Account balance can not be below minimum balance...\nCurrent bal :: {curent_bal}")
    
    # function to log transaction details to transaction table
    def log_transaction(self,date,withdraw_amount):
        try:
            conn.connect()
            cursor.reset()
            result = cursor.execute(f"SELECT * FROM bank_account WHERE UserID='{self.userid}'")
            bank_acc_id = cursor.fetchone()[2]
            cursor.execute(f"INSERT INTO transaction_table(tansaction_date,UserID,Bank_AC_id,withdrwan_amnt) VALUES('{date}','{self.userid}','{bank_acc_id}','{withdraw_amount}')")
            conn.commit()
            conn.close()
        except Exception as e:
            print(e)
            
    # function to check account statement
    def check_account_statement(self,from_date,to_date):
        try:
            conn.connect()
            trans = cursor.execute(f"SELECT * FROM transaction_table WHERE UserID = '{self.userid}' AND tansaction_date BETWEEN '{from_date}' AND '{to_date}'")
            result = cursor.fetchall()
            conn.close()
            return [i for i in result]
        except Exception as e:
            print(e)

In [14]:
a = assign(3)

In [15]:
a.check_account_bal()

1547860.0

In [16]:
a.withdraw(30)

Transaction successfull!!!
Current bal :: 1547830.0


In [17]:
s = '2000-06-01'
from_d = datetime.strptime(s,"%Y-%m-%d").date()
v = '2025-06-01'
to_d = datetime.strptime(v,"%Y-%m-%d").date()

a.check_account_statement(from_d,to_d)

[(datetime.date(2022, 6, 5), 3, '123546', 30.0)]

In [18]:
result = a.check_account_statement(from_d,to_d)
for i in result:
    print(f"Date:{i[0]}, Bank Account ID:{i[2]}, Withdraw Amount: {i[-1]}")

Date:2022-06-05, Bank Account ID:123546, Withdraw Amount: 30.0
